In [71]:
#Importing all necessary packages and setting for this project
import os
import time
import requests as req
from bs4 import BeautifulSoup ,Comment
import pandas as pd
import unicodedata
import re
pd.options.display.max_columns = 50
pd.options.display.max_rows = 50

In [72]:
#Merging data from multiple sources requires a lot of cleaning and data manipulation.
#We need to unify all the names by removing the accents and special characters
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    decode = only_ascii.decode("utf-8")
    return decode

In [73]:
#We need to remove the Sr., Jr., III, etc. from the names
#We can do this by using a regex to find the last word in the string
#and then removing it if it is in the list of suffixes
suffixes = ["Sr.","Jr.","III","II","IV","V","VI","VII","VIII","IX","X"]
def remove_suffixes(name):
    if name.split()[-1] in suffixes:
        return " ".join(name.split()[:-1])
    else:
        return name

df["Player"] = df["Player"].apply(remove_suffixes).str.replace("\.","",regex=True)
df

,Player
0,Nikola Vucevic
1,Nikola Jokic
2,Xavier Tilllman
3,Sricharan
4,God
5,IIIlman Tilller
6,PJ Dozier


In [74]:
test = ["Nikola Vučević","Nikola Jokić","Xavier Tilllman Sr.","Sricharan Jr.","God III","IIIlman Tilller","P.J. Dozier"]
df = pd.DataFrame(test,columns=["Player"])
df["Player"] = df["Player"].apply(remove_accents).apply(remove_suffixes).str.replace("\.","",regex =True)
df

,Player
0,Nikola Vucevic
1,Nikola Jokic
2,Xavier Tilllman
3,Sricharan
4,God
5,IIIlman Tilller
6,PJ Dozier


In [78]:
dpoys = pd.read_csv('/Users/david/Documents/Python Projects/DPOY/DPOYs/dpoys.csv')
dpoys["Player"] = dpoys["Player"].apply(remove_accents).apply(remove_suffixes).str.replace("\.","",regex =True)

In [79]:
#We are going to delete the Unnamed column
del dpoys['Unnamed: 0']
#We are going to make the Rank column the index
dpoys.set_index('Rank', inplace=False)
dpoys.head(25)

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
0,1,Dennis Rodman,29,DET,51.0,51.0,96,0.531,82,33.5,8.2,12.5,1.0,0.8,0.7,0.493,0.200,0.631,8.7,0.151,1991
1,2,David Robinson,25,SAS,18.0,18.0,96,0.188,82,37.7,25.6,13.0,2.5,1.5,3.9,0.552,0.143,0.762,17.0,0.264,1991
2,3,Alvin Robertson,28,MIL,15.0,15.0,96,0.156,81,32.1,13.6,5.7,5.5,3.0,0.2,0.485,0.365,0.757,7.7,0.142,1991
3,4,Joe Dumars,27,DET,3.0,3.0,96,0.031,80,38.1,20.4,2.3,5.5,1.1,0.1,0.481,0.311,0.890,9.9,0.155,1991
4,5T,Hakeem Olajuwon,28,HOU,2.0,2.0,96,0.021,56,36.8,21.2,13.8,2.3,2.2,3.9,0.508,0.000,0.769,8.6,0.201,1991
5,5T,Buck Williams,30,POR,2.0,2.0,96,0.021,80,32.3,11.7,9.4,1.2,0.6,0.6,0.602,NaN,0.705,8.5,0.158,1991
6,7T,Patrick Ewing,28,NYK,1.0,1.0,96,0.010,81,38.3,26.6,11.2,3.0,1.0,3.2,0.514,0.000,0.745,10.0,0.155,1991
7,7T,Derek Harper,29,DAL,1.0,1.0,96,0.010,77,37.4,19.7,3.0,7.1,1.9,0.2,0.467,0.362,0.731,7.6,0.127,1991
8,7T,Michael Jordan,27,CHI,1.0,1.0,96,0.010,82,37.0,31.5,6.0,5.5,2.7,1.0,0.539,0.312,0.851,20.3,0.321,1991
9,7T,Scottie Pippen,25,CHI,1.0,1.0,96,0.010,82,36.8,17.8,7.3,6.2,2.4,1.1,0.520,0.309,0.706,11.2,0.179,1991


In [80]:
#Cutting out unnecessary columns
dpoys = dpoys[["Rank","Player","Share","WS","WS/48","Year"]]
dpoys.head()

,Rank,Player,Share,WS,WS/48,Year
0,1,Dennis Rodman,0.531,8.7,0.151,1991
1,2,David Robinson,0.188,17.0,0.264,1991
2,3,Alvin Robertson,0.156,7.7,0.142,1991
3,4,Joe Dumars,0.031,9.9,0.155,1991
4,5T,Hakeem Olajuwon,0.021,8.6,0.201,1991


In [81]:
#Time to load our next csv file players.csv
players = pd.read_csv('/Users/david/Documents/Python Projects/DPOY/Players/players.csv')
players["Player"] = players["Player"].apply(remove_accents).apply(remove_suffixes).str.replace("\.","",regex =True)
players.head()

,Unnamed: 0,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,0,1,Alaa Abdelnaby,PF,22,POR,43,0,6.7,1.3,2.7,.474,0.0,0.0,NaN,1.3,2.7,.474,.474,0.6,1.0,.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1991
1,1,2,Mahmoud Abdul-Rauf,PG,21,DEN,67,19,22.5,6.2,15.1,.413,0.4,1.5,.240,5.9,13.6,.432,.425,1.3,1.5,.857,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1991
2,2,3,Mark Acres,C,28,ORL,68,0,19.3,1.6,3.1,.509,0.0,0.0,.333,1.6,3.1,.512,.512,1.0,1.5,.653,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1991
3,3,4,Michael Adams,PG,28,DEN,66,66,35.5,8.5,21.5,.394,2.5,8.5,.296,6.0,13.0,.459,.453,7.0,8.0,.879,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1991
4,4,5,Mark Aguirre,SF,31,DET,78,13,25.7,5.4,11.7,.462,0.3,1.0,.308,5.1,10.7,.477,.475,3.1,4.1,.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1991


In [82]:
#we are going to delete the "Rk" column
del players['Rk']
del players['Unnamed: 0']
players.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,Alaa Abdelnaby,PF,22,POR,43,0,6.7,1.3,2.7,.474,0.0,0.0,NaN,1.3,2.7,.474,.474,0.6,1.0,.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1991
1,Mahmoud Abdul-Rauf,PG,21,DEN,67,19,22.5,6.2,15.1,.413,0.4,1.5,.240,5.9,13.6,.432,.425,1.3,1.5,.857,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1991
2,Mark Acres,C,28,ORL,68,0,19.3,1.6,3.1,.509,0.0,0.0,.333,1.6,3.1,.512,.512,1.0,1.5,.653,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1991
3,Michael Adams,PG,28,DEN,66,66,35.5,8.5,21.5,.394,2.5,8.5,.296,6.0,13.0,.459,.453,7.0,8.0,.879,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1991
4,Mark Aguirre,SF,31,DET,78,13,25.7,5.4,11.7,.462,0.3,1.0,.308,5.1,10.7,.477,.475,3.1,4.1,.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1991


In [83]:
#Typically when dealing with these types of data sets, we will need to make sure te "Player" column is unique
#Let's check the "Player" column to see if there are any duplicates
players["Player"].head(50)
#It appears that Players that have MVP considerations have an asterisk next to their name

0         Alaa Abdelnaby
1     Mahmoud Abdul-Rauf
2             Mark Acres
3          Michael Adams
4           Mark Aguirre
5            Danny Ainge
6            Mark Alarie
7           Steve Alford
8          Greg Anderson
9          Greg Anderson
10         Greg Anderson
11         Greg Anderson
12         Nick Anderson
13          Ron Anderson
14       Willie Anderson
15        Michael Ansley
16          BJ Armstrong
17         Vincent Askew
18          Keith Askins
19           Milos Babic
20          Thurl Bailey
21           Cedric Ball
22         Ken Bannister
23      Charles Barkley*
24           Dana Barros
25           John Battle
26          Kenny Battle
27          Kenny Battle
28          Kenny Battle
29       William Bedford
30       Benoit Benjamin
31       Benoit Benjamin
32       Benoit Benjamin
33       Winston Bennett
34           Larry Bird*
35      Rolando Blackman
36          Lance Blanks
37       Mookie Blaylock
38         Muggsy Bogues
39            Manute Bol


In [84]:
#Let's remove the asterisk from the "Player" column
players["Player"] = players["Player"].str.replace("*", "",regex=False)

In [85]:
#Unfortuantely, there are still some duplicates due to players that have been traded during the regular season
#We are going to have to write a function that will combine all the rows for a player that has been traded

def single_row(df):
    if df.shape[0]==1:
        return df
    else:
        row = df[df["Tm"]=="TOT"]
        row["Tm"] = df.iloc[-1,:]["Tm"]
        return row

players = players.groupby(["Player","Year"]).apply(single_row)

In [86]:
players.head()

Player Pos Age   Tm   G  GS    MP   FG   FGA   FG%   3P  \
Player   Year                                                                   
AC Green 1991 164   AC Green  PF  27  LAL  82  21  26.4  3.1   6.6  .476  0.1   
         1992 633   AC Green  PF  28  LAL  82  53  35.4  4.7   9.8  .476  0.1   
         1993 1092  AC Green  PF  29  LAL  82  55  34.4  4.6   8.6  .537  0.2   
         1994 1579  AC Green  PF  30  PHO  82  55  34.5  5.7  11.3  .502  0.1   
         1995 2067  AC Green  SF  31  PHO  82  52  32.8  3.8   7.5  .504  0.5   

                    3PA   3P%   2P   2PA   2P%  eFG%   FT  FTA   FT%  ORB  \
Player   Year                                                               
AC Green 1991 164   0.7  .200  3.0   5.9  .507  .486  2.7  3.7  .738  2.5   
         1992 633   0.7  .214  4.5   9.1  .495  .483  4.1  5.6  .744  3.7   
         1993 1092  0.6  .348  4.4   8.0  .550  .548  3.4  4.6  .739  3.5   
         1994 1579  0.4  .229  5.6  10.9  .513  .506  3.2  4.4  .735  3.4   
         1995 2067  1.5  .339  3.3   6.0  .547  .539  3.1  4.2  .732  2.4   

                    DRB  TRB  AST  STL  BLK  TOV   PF   PTS  Year  
Player   Year                                                      
AC Green 1991 164   3.8  6.3  0.9  0.7  0.3  1.2  1.4   9.1  1991  
         1992 633   5.6  9.3  1.4  1.1  0.4  1.4  1.7  13.6  1992  
         1993 1092  5.2  8.7  1.4  1.1  0.5  1.4  1.8  12.8  1993  
         1994 1579  5.8  9.2  1.7  0.9  0.5  1.2  1.7  14.7  1994  
         1995 2067  5.8  8.2  1.5  0.7  0.4  1.4  1.8  11.2  1995

In [87]:
#We have to get rid of the multi-index twice and normalize the index
players.index = players.index.droplevel()

In [88]:
players.index = players.index.droplevel()
players

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
164,AC Green,PF,27,LAL,82,21,26.4,3.1,6.6,.476,0.1,0.7,.200,3.0,5.9,.507,.486,2.7,3.7,.738,2.5,3.8,6.3,0.9,0.7,0.3,1.2,1.4,9.1,1991
633,AC Green,PF,28,LAL,82,53,35.4,4.7,9.8,.476,0.1,0.7,.214,4.5,9.1,.495,.483,4.1,5.6,.744,3.7,5.6,9.3,1.4,1.1,0.4,1.4,1.7,13.6,1992
1092,AC Green,PF,29,LAL,82,55,34.4,4.6,8.6,.537,0.2,0.6,.348,4.4,8.0,.550,.548,3.4,4.6,.739,3.5,5.2,8.7,1.4,1.1,0.5,1.4,1.8,12.8,1993
1579,AC Green,PF,30,PHO,82,55,34.5,5.7,11.3,.502,0.1,0.4,.229,5.6,10.9,.513,.506,3.2,4.4,.735,3.4,5.8,9.2,1.7,0.9,0.5,1.2,1.7,14.7,1994
2067,AC Green,SF,31,PHO,82,52,32.8,3.8,7.5,.504,0.5,1.5,.339,3.3,6.0,.547,.539,3.1,4.2,.732,2.4,5.8,8.2,1.5,0.7,0.4,1.4,1.8,11.2,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9943,Zydrunas Ilgauskas,C,33,CLE,65,65,27.2,5.3,11.1,.472,0.2,0.6,.385,5.0,10.5,.477,.483,2.1,2.7,.799,2.4,5.1,7.5,1.0,0.4,1.3,1.4,2.8,12.9,2009
10551,Zydrunas Ilgauskas,C,34,CLE,64,6,20.9,3.0,6.8,.443,0.2,0.4,.478,2.9,6.5,.441,.455,1.2,1.6,.743,1.8,3.6,5.4,0.8,0.2,0.8,1.0,2.9,7.4,2010
11186,Zydrunas Ilgauskas,C,35,MIA,72,51,15.9,2.3,4.4,.508,0.0,0.0,.000,2.3,4.4,.509,.508,0.5,0.6,.783,1.5,2.5,4.0,0.4,0.3,0.8,0.7,2.6,5.0,2011
16762,Zylan Cheatham,PF,24,NOP,4,0,12.8,1.5,2.3,.667,0.0,0.3,.000,1.5,2.0,.750,.667,0.0,0.0,NaN,0.8,1.5,2.3,0.8,0.3,0.3,1.0,2.5,3.0,2020


In [89]:
#Lastly, let's resort based on the index values
players.sort_index(inplace=True)

In [91]:
#Merging players and dpoys tables
dpoy_players = pd.merge(dpoys, players, how="outer", on=["Player","Year"])
dpoy_players.tail()

,Rank,Player,Share,WS,WS/48,Year,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
14692,NaN,Thaddeus Young,NaN,NaN,NaN,2022,PF,33,TOR,52,1,16.3,2.7,5.2,.518,0.3,0.9,.354,2.4,4.3,.554,.550,0.4,0.9,.469,1.5,2.5,4.0,2.0,1.0,0.3,1.0,1.6,6.2
14693,NaN,Trae Young,NaN,NaN,NaN,2022,PG,23,ATL,76,76,34.9,9.4,20.3,.460,3.1,8.0,.382,6.3,12.3,.512,.536,6.6,7.3,.904,0.7,3.1,3.7,9.7,0.9,0.1,4.0,1.7,28.4
14694,NaN,Omer Yurtseven,NaN,NaN,NaN,2022,C,23,MIA,56,12,12.6,2.3,4.4,.526,0.0,0.2,.091,2.3,4.2,.547,.528,0.7,1.1,.623,1.5,3.7,5.3,0.9,0.3,0.4,0.7,1.5,5.3
14695,NaN,Cody Zeller,NaN,NaN,NaN,2022,C,29,POR,27,0,13.1,1.9,3.3,.567,0.0,0.1,.000,1.9,3.2,.593,.567,1.4,1.8,.776,1.9,2.8,4.6,0.8,0.3,0.2,0.7,2.1,5.2
14696,NaN,Ivica Zubac,NaN,NaN,NaN,2022,C,24,LAC,76,76,24.4,4.1,6.5,.626,0.0,0.0,NaN,4.1,6.5,.626,.626,2.2,3.0,.727,2.9,5.6,8.5,1.6,0.5,1.0,1.5,2.7,10.3


In [92]:
#Replacing all null values with 0
dpoy_players.fillna(0, inplace=True)
dpoy_players.tail()

,Rank,Player,Share,WS,WS/48,Year,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
14692,0,Thaddeus Young,0.0,0.0,0.0,2022,PF,33,TOR,52,1,16.3,2.7,5.2,.518,0.3,0.9,.354,2.4,4.3,.554,.550,0.4,0.9,.469,1.5,2.5,4.0,2.0,1.0,0.3,1.0,1.6,6.2
14693,0,Trae Young,0.0,0.0,0.0,2022,PG,23,ATL,76,76,34.9,9.4,20.3,.460,3.1,8.0,.382,6.3,12.3,.512,.536,6.6,7.3,.904,0.7,3.1,3.7,9.7,0.9,0.1,4.0,1.7,28.4
14694,0,Omer Yurtseven,0.0,0.0,0.0,2022,C,23,MIA,56,12,12.6,2.3,4.4,.526,0.0,0.2,.091,2.3,4.2,.547,.528,0.7,1.1,.623,1.5,3.7,5.3,0.9,0.3,0.4,0.7,1.5,5.3
14695,0,Cody Zeller,0.0,0.0,0.0,2022,C,29,POR,27,0,13.1,1.9,3.3,.567,0.0,0.1,.000,1.9,3.2,.593,.567,1.4,1.8,.776,1.9,2.8,4.6,0.8,0.3,0.2,0.7,2.1,5.2
14696,0,Ivica Zubac,0.0,0.0,0.0,2022,C,24,LAC,76,76,24.4,4.1,6.5,.626,0.0,0.0,0,4.1,6.5,.626,.626,2.2,3.0,.727,2.9,5.6,8.5,1.6,0.5,1.0,1.5,2.7,10.3


In [93]:
#Let's import our next csv file, historical_Raptor_by_player.csv
raptor = pd.read_csv('/Users/david/Documents/Python Projects/DPOY/Players/historical_Raptor_by_player.csv')
raptor["player_name"] = raptor["player_name"].apply(remove_accents).apply(remove_suffixes).str.replace("\.","",regex =True)
raptor.head()

,player_name,player_id,season,poss,mp,raptor_offense,raptor_defense,raptor_total,war_total,war_reg_season,war_playoffs,predator_offense,predator_defense,predator_total,pace_impact
0,Alaa Abdelnaby,abdelal01,1991,640,303,-3.938450,-0.510076,-4.448526,-0.265191,-0.226163,-0.039028,-3.905732,-1.696796,-5.602528,0.199241
1,Alaa Abdelnaby,abdelal01,1992,1998,959,-2.553849,-0.197943,-2.751792,-0.006893,0.070322,-0.077215,-2.499576,-0.721876,-3.221452,0.485436
2,Alaa Abdelnaby,abdelal01,1993,2754,1379,-2.373736,-2.069808,-4.443544,-1.227189,-0.845272,-0.381918,-2.328144,-2.002263,-4.330407,0.488266
3,Alaa Abdelnaby,abdelal01,1994,320,159,-6.140056,-2.748312,-8.888368,-0.500600,-0.500600,0.000000,-5.820936,-3.526560,-9.347496,0.511152
4,Alaa Abdelnaby,abdelal01,1995,984,506,-3.846543,-1.268012,-5.114556,-0.615802,-0.615802,0.000000,-3.715429,-1.027024,-4.742453,1.545922


In [94]:
#Let's delete the following columns: player_id,raptor_offense,war_playoffs,predator_offense
raptor.drop(columns=['player_id','raptor_offense','war_playoffs','predator_offense'], inplace=True)
raptor.head()

,player_name,season,poss,mp,raptor_defense,raptor_total,war_total,war_reg_season,predator_defense,predator_total,pace_impact
0,Alaa Abdelnaby,1991,640,303,-0.510076,-4.448526,-0.265191,-0.226163,-1.696796,-5.602528,0.199241
1,Alaa Abdelnaby,1992,1998,959,-0.197943,-2.751792,-0.006893,0.070322,-0.721876,-3.221452,0.485436
2,Alaa Abdelnaby,1993,2754,1379,-2.069808,-4.443544,-1.227189,-0.845272,-2.002263,-4.330407,0.488266
3,Alaa Abdelnaby,1994,320,159,-2.748312,-8.888368,-0.500600,-0.500600,-3.526560,-9.347496,0.511152
4,Alaa Abdelnaby,1995,984,506,-1.268012,-5.114556,-0.615802,-0.615802,-1.027024,-4.742453,1.545922


In [95]:
#Let's eliminate the rows that are before season 1991-1992
raptor = raptor[raptor["season"]>1990]
raptor.tail()

,player_name,season,poss,mp,raptor_defense,raptor_total,war_total,war_reg_season,predator_defense,predator_total,pace_impact
18548,Ivica Zubac,2017,1305,609,-3.040589,-6.398767,-1.141160,-1.141160,-2.932103,-6.465707,0.644128
18549,Ivica Zubac,2018,871,410,2.688832,-0.214877,0.531968,0.531968,0.752875,-1.863456,0.311576
18550,Ivica Zubac,2019,2345,1079,1.813768,-0.548676,1.217134,1.114995,1.157824,-1.652459,0.077074
18551,Ivica Zubac,2020,3447,1646,3.549458,4.649308,6.224207,5.020903,3.331974,3.021132,-0.410164
18552,Ivica Zubac,2021,3908,1910,2.525735,1.616696,4.261327,3.704630,2.280011,1.405673,-0.117623


In [96]:
#Matching columns to our next df for concatenation
column_list = []
for column in raptor.columns:
    column_list.append(column)
print(column_list)

['player_name', 'season', 'poss', 'mp', 'raptor_defense', 'raptor_total', 'war_total', 'war_reg_season', 'predator_defense', 'predator_total', 'pace_impact']


In [97]:
#The previous raptor csv did not include the most recent 2022 season, so we will clean it up a bit and merge it with the previous raptor csv
raptor_2022 = pd.read_csv('/Users/david/Documents/Python Projects/DPOY/Players/latest_RAPTOR_by_player.csv')
raptor_2022["player_name"] = raptor_2022["player_name"].apply(remove_accents).apply(remove_suffixes).str.replace("\.","",regex =True)
raptor_2022.tail()

,player_name,player_id,season,poss,mp,raptor_box_offense,raptor_box_defense,raptor_box_total,raptor_onoff_offense,raptor_onoff_defense,raptor_onoff_total,raptor_offense,raptor_defense,raptor_total,war_total,war_reg_season,war_playoffs,predator_offense,predator_defense,predator_total,pace_impact
601,Thaddeus Young,youngth01,2022,1927,932,-0.232464,0.330621,0.098157,-1.608523,2.074779,0.466257,-0.540097,0.726837,0.186740,1.417299,1.447940,-0.030641,-0.490506,1.012304,0.521798,0.412187
602,Trae Young,youngtr01,2022,5903,2838,7.064969,-2.762787,4.302181,3.647055,-3.381590,0.265465,6.715589,-3.049648,3.665941,9.403349,9.903858,-0.500510,6.523703,-3.356965,3.166738,2.214593
603,Omer Yurtseven,yurtsom01,2022,1525,744,-2.696110,1.256200,-1.439910,1.337087,-0.584524,0.752563,-2.049417,0.876685,-1.172733,0.599452,0.563127,0.036325,-2.256519,0.176608,-2.079910,-0.035254
604,Cody Zeller,zelleco01,2022,748,355,-0.380332,-3.771124,-4.151456,0.365332,-0.402120,-0.036788,-0.222446,-3.226756,-3.449202,-0.127035,-0.127035,0.000000,-0.396488,-2.401036,-2.797523,0.312001
605,Ivica Zubac,zubaciv01,2022,3786,1852,-0.920033,1.986362,1.066330,0.343447,-4.677449,-4.334003,-0.693765,0.723358,0.029594,2.626958,2.626958,0.000000,-1.134139,1.759848,0.625709,0.020823


In [98]:
#Let's match these columns to our previous raptor df
raptor_2022 = raptor_2022[column_list]
raptor_2022.tail()

,player_name,season,poss,mp,raptor_defense,raptor_total,war_total,war_reg_season,predator_defense,predator_total,pace_impact
601,Thaddeus Young,2022,1927,932,0.726837,0.186740,1.417299,1.447940,1.012304,0.521798,0.412187
602,Trae Young,2022,5903,2838,-3.049648,3.665941,9.403349,9.903858,-3.356965,3.166738,2.214593
603,Omer Yurtseven,2022,1525,744,0.876685,-1.172733,0.599452,0.563127,0.176608,-2.079910,-0.035254
604,Cody Zeller,2022,748,355,-3.226756,-3.449202,-0.127035,-0.127035,-2.401036,-2.797523,0.312001
605,Ivica Zubac,2022,3786,1852,0.723358,0.029594,2.626958,2.626958,1.759848,0.625709,0.020823


In [100]:
#Let's do a concatenation on raptor_2022 and raptor
raptor_cleaned = pd.concat([raptor,raptor_2022], axis=0)
raptor_cleaned.reset_index(inplace=True)
raptor_cleaned.tail(1000)

,index,player_name,season,poss,mp,raptor_defense,raptor_total,war_total,war_reg_season,predator_defense,predator_total,pace_impact
13718,18012,Lou Williams,2018,5489,2589,-3.336744,1.882814,6.274484,6.274484,-3.522228,1.759520,2.511937
13719,18013,Lou Williams,2019,4775,2169,-3.945944,1.063931,4.286623,4.304897,-4.644651,-0.770091,1.713151
13720,18014,Lou Williams,2020,4802,2205,-2.724414,-0.507715,2.576833,2.114615,-2.857787,-1.083755,1.581810
13721,18015,Lou Williams,2021,3584,1700,-3.097436,-2.787252,-0.021688,-0.294430,-3.860788,-2.630943,0.756711
13722,18016,Mo Williams,2004,1409,772,-1.572965,-4.263100,-0.599883,-0.599883,-1.877015,-4.502399,0.598451
...,...,...,...,...,...,...,...,...,...,...,...,...
14713,601,Thaddeus Young,2022,1927,932,0.726837,0.186740,1.417299,1.447940,1.012304,0.521798,0.412187
14714,602,Trae Young,2022,5903,2838,-3.049648,3.665941,9.403349,9.903858,-3.356965,3.166738,2.214593
14715,603,Omer Yurtseven,2022,1525,744,0.876685,-1.172733,0.599452,0.563127,0.176608,-2.079910,-0.035254
14716,604,Cody Zeller,2022,748,355,-3.226756,-3.449202,-0.127035,-0.127035,-2.401036,-2.797523,0.312001


In [101]:
#Delte index column
del raptor_cleaned['index']
del raptor_cleaned["mp"]
raptor_cleaned.tail()

,player_name,season,poss,raptor_defense,raptor_total,war_total,war_reg_season,predator_defense,predator_total,pace_impact
14713,Thaddeus Young,2022,1927,0.726837,0.186740,1.417299,1.447940,1.012304,0.521798,0.412187
14714,Trae Young,2022,5903,-3.049648,3.665941,9.403349,9.903858,-3.356965,3.166738,2.214593
14715,Omer Yurtseven,2022,1525,0.876685,-1.172733,0.599452,0.563127,0.176608,-2.079910,-0.035254
14716,Cody Zeller,2022,748,-3.226756,-3.449202,-0.127035,-0.127035,-2.401036,-2.797523,0.312001
14717,Ivica Zubac,2022,3786,0.723358,0.029594,2.626958,2.626958,1.759848,0.625709,0.020823


In [102]:
#We need to change some column names around to merge later on. 
#We will change the "season" column to "Year" and the "player_name" column to "Player"
raptor_cleaned.rename(columns={"season":"Year","player_name":"Player"}, inplace=True)
raptor_cleaned.tail()

,Player,Year,poss,raptor_defense,raptor_total,war_total,war_reg_season,predator_defense,predator_total,pace_impact
14713,Thaddeus Young,2022,1927,0.726837,0.186740,1.417299,1.447940,1.012304,0.521798,0.412187
14714,Trae Young,2022,5903,-3.049648,3.665941,9.403349,9.903858,-3.356965,3.166738,2.214593
14715,Omer Yurtseven,2022,1525,0.876685,-1.172733,0.599452,0.563127,0.176608,-2.079910,-0.035254
14716,Cody Zeller,2022,748,-3.226756,-3.449202,-0.127035,-0.127035,-2.401036,-2.797523,0.312001
14717,Ivica Zubac,2022,3786,0.723358,0.029594,2.626958,2.626958,1.759848,0.625709,0.020823


In [103]:
#Let's save raptor_cleaned as a csv file in our Cleaned folder
raptor_cleaned.to_csv('/Users/david/Documents/Python Projects/DPOY/Cleaned/raptor_cleaned.csv')

In [104]:
#Time to load the teams.csv file and eventually merge it with our dpoy_players df
teams = pd.read_csv('/Users/david/Documents/Python Projects/DPOY/Team_Records/teams.csv')
del teams['Unnamed: 0']
teams.head()

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,56,26,.683,—,111.5,105.7,5.22,1991,Boston Celtics*
1,44,38,.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers*
2,39,43,.476,17.0,103.1,103.3,-0.43,1991,New York Knicks*
3,30,52,.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets
4,26,56,.317,30.0,102.9,107.5,-4.53,1991,New Jersey Nets


In [105]:
#Clean up the "Team" column and removing the * from the end of the team name
teams["Team"] = teams["Team"].str.replace("*", "",regex=False)
teams.head(20)

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,56,26,.683,—,111.5,105.7,5.22,1991,Boston Celtics
1,44,38,.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers
2,39,43,.476,17.0,103.1,103.3,-0.43,1991,New York Knicks
3,30,52,.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets
4,26,56,.317,30.0,102.9,107.5,-4.53,1991,New Jersey Nets
5,24,58,.293,32.0,101.8,107.8,-5.91,1991,Miami Heat
6,Central Division,Central Division,Central Division,Central Division,Central Division,Central Division,Central Division,1991,Central Division
7,61,21,.744,—,110.0,101.0,8.57,1991,Chicago Bulls
8,50,32,.610,11.0,100.1,96.8,3.08,1991,Detroit Pistons
9,48,34,.585,13.0,106.4,104.0,2.33,1991,Milwaukee Bucks


In [106]:
#We need to remove every row with the a division name
teams = teams[~teams["W"].str.contains("Division")].copy()
teams.head(20)

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,56,26,.683,—,111.5,105.7,5.22,1991,Boston Celtics
1,44,38,.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers
2,39,43,.476,17.0,103.1,103.3,-0.43,1991,New York Knicks
3,30,52,.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets
4,26,56,.317,30.0,102.9,107.5,-4.53,1991,New Jersey Nets
5,24,58,.293,32.0,101.8,107.8,-5.91,1991,Miami Heat
7,61,21,.744,—,110.0,101.0,8.57,1991,Chicago Bulls
8,50,32,.610,11.0,100.1,96.8,3.08,1991,Detroit Pistons
9,48,34,.585,13.0,106.4,104.0,2.33,1991,Milwaukee Bucks
10,43,39,.524,18.0,109.8,109.0,0.72,1991,Atlanta Hawks


In [107]:
sorted(teams["Team"].unique())

['Atlanta Hawks',
 'Boston Celtics',
 'Brooklyn Nets',
 'Charlotte Bobcats',
 'Charlotte Hornets',
 'Chicago Bulls',
 'Cleveland Cavaliers',
 'Dallas Mavericks',
 'Denver Nuggets',
 'Detroit Pistons',
 'Golden State Warriors',
 'Houston Rockets',
 'Indiana Pacers',
 'Los Angeles Clippers',
 'Los Angeles Lakers',
 'Memphis Grizzlies',
 'Miami Heat',
 'Milwaukee Bucks',
 'Minnesota Timberwolves',
 'New Jersey Nets',
 'New Orleans Hornets',
 'New Orleans Pelicans',
 'New Orleans/Oklahoma City Hornets',
 'New York Knicks',
 'Oklahoma City Thunder',
 'Orlando Magic',
 'Philadelphia 76ers',
 'Phoenix Suns',
 'Portland Trail Blazers',
 'Sacramento Kings',
 'San Antonio Spurs',
 'Seattle SuperSonics',
 'Toronto Raptors',
 'Utah Jazz',
 'Vancouver Grizzlies',
 'Washington Bullets',
 'Washington Wizards']

In [108]:
sorted(players["Tm"].unique())

['ATL',
 'BOS',
 'BRK',
 'CHA',
 'CHH',
 'CHI',
 'CHO',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GSW',
 'HOU',
 'IND',
 'LAC',
 'LAL',
 'MEM',
 'MIA',
 'MIL',
 'MIN',
 'NJN',
 'NOH',
 'NOK',
 'NOP',
 'NYK',
 'OKC',
 'ORL',
 'PHI',
 'PHO',
 'POR',
 'SAC',
 'SAS',
 'SEA',
 'TOR',
 'UTA',
 'VAN',
 'WAS',
 'WSB']

In [109]:
#We will need to make a csv file with the "Tm" name in column 1 from players dataframe and the full team name in column 2 from teams dataframe. 
#First we need zip the two lists together"Abbreviated":
team_df = pd.DataFrame(list(zip(sorted(players["Tm"].unique()),sorted(teams["Team"].unique()))))
team_df.head()

,0,1
0,ATL,Atlanta Hawks
1,BOS,Boston Celtics
2,BRK,Brooklyn Nets
3,CHA,Charlotte Bobcats
4,CHH,Charlotte Hornets


In [110]:
#Let's save this as a csv under the Team_Records folder
team_df.to_csv('/Users/david/Documents/Python Projects/DPOY/Team_Records/team_map.csv')
#I manually altered this short list to map out to the correct full team name

In [111]:
#Time to turn the team_map.csv into a dictionary
team_map = pd.read_csv('/Users/david/Documents/Python Projects/DPOY/Team_Records/team_map.csv')
#Rename first column as "Abbreviated"
team_map.rename(columns={"0":"Abbreviated"}, inplace=True)
#Rename second column as "Full Team Name"
team_map.rename(columns={"1":"Full Team Name"}, inplace=True)
#Save csv file
team_map.to_csv('/Users/david/Documents/Python Projects/DPOY/Team_Records/team_map.csv')


In [116]:
#turn into dictionary
team_map=pd.read_csv('/Users/david/Documents/Python Projects/DPOY/Team_Records/team_map.csv')
team_map_dict = dict(zip(team_map["Abbreviated"],team_map["Full Team Name"]))

In [117]:
team_map_dict

{'ATL': 'Atlanta Hawks',
 'BOS': 'Boston Celtics',
 'BRK': 'Brooklyn Nets',
 'CHA': 'Charlotte Bobcats',
 'CHH': 'Charlotte Hornets',
 'CHO': 'Charlotte Hornets',
 'CHI': 'Chicago Bulls',
 'CLE': 'Cleveland Cavaliers',
 'DAL': 'Dallas Mavericks',
 'DEN': 'Denver Nuggets',
 'DET': 'Detroit Pistons',
 'GSW': 'Golden State Warriors',
 'HOU': 'Houston Rockets',
 'IND': 'Indiana Pacers',
 'LAC': 'Los Angeles Clippers',
 'LAL': 'Los Angeles Lakers',
 'MEM': 'Memphis Grizzlies',
 'MIA': 'Miami Heat',
 'MIL': 'Milwaukee Bucks',
 'MIN': 'Minnesota Timberwolves',
 'NJN': 'New Jersey Nets',
 'NOH': 'New Orleans Hornets',
 'NOP': 'New Orleans Pelicans',
 'NOK': 'New Orleans/Oklahoma City Hornets',
 'NYK': 'New York Knicks',
 'OKC': 'Oklahoma City Thunder',
 'ORL': 'Orlando Magic',
 'PHI': 'Philadelphia 76ers',
 'PHO': 'Phoenix Suns',
 'POR': 'Portland Trail Blazers',
 'SAC': 'Sacramento Kings',
 'SAS': 'San Antonio Spurs',
 'SEA': 'Seattle SuperSonics',
 'TOR': 'Toronto Raptors',
 'UTA': 'Utah Jaz

In [118]:
#Let's map a new column "Team" to our dpoy_players df that maps out from the team_map_dict
dpoy_players["Team"] = dpoy_players["Tm"].map(team_map_dict)
dpoy_players.head()

,Rank,Player,Share,WS,WS/48,Year,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team
0,1,Dennis Rodman,0.531,8.7,0.151,1991,SF,29,DET,82,77,33.5,3.4,6.8,.493,0.1,0.4,.200,3.3,6.5,.509,.498,1.4,2.1,.631,4.4,8.1,12.5,1.0,0.8,0.7,1.1,3.4,8.2,Detroit Pistons
1,2,David Robinson,0.188,17.0,0.264,1991,C,25,SAS,82,81,37.7,9.2,16.7,.552,0.0,0.1,.143,9.2,16.6,.554,.552,7.2,9.5,.762,4.1,8.9,13.0,2.5,1.5,3.9,3.3,3.2,25.6,San Antonio Spurs
2,3,Alvin Robertson,0.156,7.7,0.142,1991,SG,28,MIL,81,81,32.1,5.4,11.2,.485,0.3,0.8,.365,5.1,10.4,.493,.497,2.5,3.2,.757,2.4,3.3,5.7,5.5,3.0,0.2,2.6,3.4,13.6,Milwaukee Bucks
3,4,Joe Dumars,0.031,9.9,0.155,1991,SG,27,DET,80,80,38.1,7.8,16.2,.481,0.2,0.6,.311,7.6,15.6,.488,.487,4.6,5.2,.890,0.8,1.6,2.3,5.5,1.1,0.1,2.4,1.7,20.4,Detroit Pistons
4,5T,Hakeem Olajuwon,0.021,8.6,0.201,1991,C,28,HOU,56,50,36.8,8.7,17.1,.508,0.0,0.1,.000,8.7,17.1,.510,.508,3.8,4.9,.769,3.9,9.8,13.8,2.3,2.2,3.9,3.1,3.9,21.2,Houston Rockets


In [68]:
#Let's map a new column "Tm" onto teams df that maps out from the team_map_dict
teams["Tm"] = teams["Team"].map(team_map_dict)
#Deleting "PS/G" becase that's points scored which has nothing to do with defense
#CHECK RIGHT BELOW THIS LINE
#del teams["PS/G"]
teams.head(30)

,W,L,W/L%,GB,PA/G,SRS,Year,Team,Tm
0,56,26,.683,—,105.7,5.22,1991,Boston Celtics,NaN
1,44,38,.537,12.0,105.6,-0.39,1991,Philadelphia 76ers,NaN
2,39,43,.476,17.0,103.3,-0.43,1991,New York Knicks,NaN
3,30,52,.366,26.0,106.4,-4.84,1991,NaN,NaN
4,26,56,.317,30.0,107.5,-4.53,1991,New Jersey Nets,NaN
5,24,58,.293,32.0,107.8,-5.91,1991,Miami Heat,NaN
7,61,21,.744,—,101.0,8.57,1991,Chicago Bulls,NaN
8,50,32,.610,11.0,96.8,3.08,1991,Detroit Pistons,NaN
9,48,34,.585,13.0,104.0,2.33,1991,Milwaukee Bucks,NaN
10,43,39,.524,18.0,109.0,0.72,1991,Atlanta Hawks,NaN


In [121]:
#Time to merge the teams df with the dpoy_players df
dpoy_players_teams = pd.merge(dpoy_players, teams, how="outer", on=["Year","Team"])
dpoy_players_teams.head()

,Rank,Player,Share,WS,WS/48,Year,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
0,1,Dennis Rodman,0.531,8.7,0.151,1991,SF,29,DET,82,77,33.5,3.4,6.8,.493,0.1,0.4,.200,3.3,6.5,.509,.498,1.4,2.1,.631,4.4,8.1,12.5,1.0,0.8,0.7,1.1,3.4,8.2,Detroit Pistons,50,32,.610,11.0,100.1,96.8,3.08
1,4,Joe Dumars,0.031,9.9,0.155,1991,SG,27,DET,80,80,38.1,7.8,16.2,.481,0.2,0.6,.311,7.6,15.6,.488,.487,4.6,5.2,.890,0.8,1.6,2.3,5.5,1.1,0.1,2.4,1.7,20.4,Detroit Pistons,50,32,.610,11.0,100.1,96.8,3.08
2,0,Mark Aguirre,0.000,0.0,0.000,1991,SF,31,DET,78,13,25.7,5.4,11.7,.462,0.3,1.0,.308,5.1,10.7,.477,.475,3.1,4.1,.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,Detroit Pistons,50,32,.610,11.0,100.1,96.8,3.08
3,0,William Bedford,0.000,0.0,0.000,1991,C,27,DET,60,4,9.4,1.8,4.0,.438,0.1,0.2,.385,1.7,3.8,.441,.448,0.9,1.3,.705,0.9,1.3,2.2,0.5,0.0,0.6,0.5,1.3,4.5,Detroit Pistons,50,32,.610,11.0,100.1,96.8,3.08
4,0,Lance Blanks,0.000,0.0,0.000,1991,SG,24,DET,38,0,5.6,0.7,1.6,.426,0.1,0.4,.125,0.6,1.2,.533,.443,0.3,0.4,.714,0.1,0.4,0.5,0.7,0.2,0.1,0.5,0.9,1.7,Detroit Pistons,50,32,.610,11.0,100.1,96.8,3.08


In [122]:
#Let's delete the columns: WS,WS/48 since it only came with the DPOY dataset
del dpoy_players_teams["WS"]
del dpoy_players_teams["WS/48"]
dpoy_players_teams.head()

,Rank,Player,Share,Year,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
0,1,Dennis Rodman,0.531,1991,SF,29,DET,82,77,33.5,3.4,6.8,.493,0.1,0.4,.200,3.3,6.5,.509,.498,1.4,2.1,.631,4.4,8.1,12.5,1.0,0.8,0.7,1.1,3.4,8.2,Detroit Pistons,50,32,.610,11.0,100.1,96.8,3.08
1,4,Joe Dumars,0.031,1991,SG,27,DET,80,80,38.1,7.8,16.2,.481,0.2,0.6,.311,7.6,15.6,.488,.487,4.6,5.2,.890,0.8,1.6,2.3,5.5,1.1,0.1,2.4,1.7,20.4,Detroit Pistons,50,32,.610,11.0,100.1,96.8,3.08
2,0,Mark Aguirre,0.000,1991,SF,31,DET,78,13,25.7,5.4,11.7,.462,0.3,1.0,.308,5.1,10.7,.477,.475,3.1,4.1,.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,Detroit Pistons,50,32,.610,11.0,100.1,96.8,3.08
3,0,William Bedford,0.000,1991,C,27,DET,60,4,9.4,1.8,4.0,.438,0.1,0.2,.385,1.7,3.8,.441,.448,0.9,1.3,.705,0.9,1.3,2.2,0.5,0.0,0.6,0.5,1.3,4.5,Detroit Pistons,50,32,.610,11.0,100.1,96.8,3.08
4,0,Lance Blanks,0.000,1991,SG,24,DET,38,0,5.6,0.7,1.6,.426,0.1,0.4,.125,0.6,1.2,.533,.443,0.3,0.4,.714,0.1,0.4,0.5,0.7,0.2,0.1,0.5,0.9,1.7,Detroit Pistons,50,32,.610,11.0,100.1,96.8,3.08


In [123]:
#Lets find Jeff Green in dpoy_players_teams df
dpoy_players_teams[dpoy_players_teams["Player"]=="Jeff Green"]

,Rank,Player,Share,Year,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
3019,0,Jeff Green,0.0,2010,PF,23,OKC,82,82,37.1,5.9,13.0,.453,1.3,3.8,.333,4.6,9.2,.503,.502,2.0,2.7,.740,1.4,4.6,6.0,1.6,1.3,0.9,1.6,2.7,15.1,Oklahoma City Thunder,50,32,.610,3.0,101.5,98.0,3.55
3176,0,Jeff Green,0.0,2011,PF-SF,24,BOS,75,51,32.4,4.9,10.9,.449,0.9,2.8,.303,4.1,8.1,.499,.488,2.6,3.3,.811,1.0,3.8,4.8,1.4,0.7,0.5,1.3,2.8,13.3,Boston Celtics,56,26,.683,—,96.5,91.1,4.83
3767,0,Jeff Green,0.0,2013,PF,26,BOS,81,17,27.8,4.6,10.0,.467,0.9,2.2,.385,3.8,7.7,.490,.510,2.6,3.3,.808,0.7,3.2,3.9,1.6,0.7,0.8,1.6,2.2,12.8,Boston Celtics,41,40,.506,12.5,96.5,96.7,-0.62
4113,0,Jeff Green,0.0,2015,SF,28,MEM,78,70,31.5,5.3,12.4,.430,1.2,3.7,.332,4.1,8.7,.473,.480,3.1,3.7,.833,0.8,3.4,4.2,1.7,0.7,0.4,1.4,1.9,15.0,Memphis Grizzlies,55,27,.671,1.0,98.3,95.1,3.62
4291,0,Jeff Green,0.0,2016,SF,29,LAC,80,41,28.2,4.4,10.3,.430,0.8,2.7,.315,3.6,7.6,.470,.470,2.1,2.8,.745,0.9,3.2,4.2,1.7,0.7,0.5,1.2,2.1,11.7,Los Angeles Clippers,53,29,.646,20.0,104.5,100.2,4.13
10487,0,Jeff Green,0.0,2008,SF,21,SEA,80,52,28.2,4.0,9.4,.427,0.3,1.0,.276,3.7,8.4,.444,.441,2.2,3.0,.744,1.3,3.5,4.7,1.5,0.6,0.6,2.0,2.5,10.5,Seattle SuperSonics,20,62,.244,34.0,97.5,106.3,-8.04
10618,0,Jeff Green,0.0,2009,PF,22,OKC,78,78,36.8,6.1,13.7,.446,1.2,3.2,.389,4.9,10.5,.463,.491,3.1,3.9,.788,1.5,5.1,6.7,2.0,1.0,0.4,2.2,2.6,16.5,Oklahoma City Thunder,23,59,.280,31.0,97.0,103.1,-6.04
11765,0,Jeff Green,0.0,2014,SF,27,BOS,82,82,34.2,5.9,14.3,.412,1.6,4.8,.341,4.2,9.5,.448,.469,3.5,4.3,.795,0.7,4.0,4.6,1.7,0.7,0.6,2.0,2.2,16.9,Boston Celtics,25,57,.305,23.0,96.2,100.7,-4.97
12637,0,Jeff Green,0.0,2017,PF,30,ORL,69,11,22.2,3.2,8.1,.394,0.8,2.8,.275,2.4,5.3,.458,.442,2.1,2.4,.863,0.6,2.5,3.1,1.2,0.5,0.2,1.1,1.5,9.2,Orlando Magic,29,53,.354,20.0,101.1,107.6,-6.61
13141,0,Jeff Green,0.0,2018,PF,31,CLE,78,13,23.4,3.8,7.9,.477,0.7,2.2,.312,3.1,5.7,.540,.520,2.6,3.0,.868,0.8,2.4,3.2,1.3,0.5,0.4,1.0,1.9,10.8,Cleveland Cavaliers,50,32,.610,—,110.9,109.9,0.59


In [124]:
#Lastly, let's merge one more time with the raptor_cleaned df
completed_df = pd.merge(dpoy_players_teams, raptor_cleaned, how="outer", on=["Player","Year"])
completed_df.tail(500)

,Rank,Player,Share,Year,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,poss,raptor_defense,raptor_total,war_total,war_reg_season,predator_defense,predator_total,pace_impact
14282,0,Andre Drummond,0.0,2022,C,28,BRK,73,36,19.7,3.4,5.9,.570,0.0,0.0,.000,3.4,5.9,.574,.570,1.2,2.2,.524,3.1,6.2,9.3,1.8,1.1,0.9,1.6,2.6,7.9,Brooklyn Nets,44,38,.537,7.0,112.9,112.1,0.82,3092.0,2.351630,1.905929,3.575294,3.497033,2.977973,2.440432,0.592110
14283,0,David Duke,0.0,2022,PG,22,BRK,22,7,15.5,1.8,4.9,.361,0.4,1.7,.243,1.4,3.2,.423,.403,0.8,1.0,.810,1.4,1.7,3.0,0.8,0.6,0.3,0.4,1.6,4.7,Brooklyn Nets,44,38,.537,7.0,112.9,112.1,0.82,714.0,-1.643254,-3.242686,-0.085782,-0.085782,-0.777155,-2.518287,0.076653
14284,0,Kevin Durant,0.0,2022,PF,33,BRK,55,55,37.2,10.5,20.3,.518,2.1,5.5,.383,8.4,14.8,.568,.570,6.8,7.4,.910,0.5,6.9,7.4,6.4,0.9,0.9,3.5,2.1,29.9,Brooklyn Nets,44,38,.537,7.0,112.9,112.1,0.82,4667.0,0.052727,4.501900,8.250891,8.916097,-0.904172,4.235432,1.891119
14285,0,Kessler Edwards,0.0,2022,SF,21,BRK,48,23,20.6,2.3,5.6,.412,1.0,2.8,.353,1.3,2.7,.473,.502,0.3,0.4,.842,0.9,2.7,3.6,0.6,0.6,0.5,0.9,1.8,5.9,Brooklyn Nets,44,38,.537,7.0,112.9,112.1,0.82,2086.0,-1.864445,-3.888945,-0.579428,-0.565793,-1.518767,-3.459559,0.193073
14286,0,Blake Griffin,0.0,2022,C,32,BRK,56,24,17.1,2.4,5.6,.425,0.7,2.6,.262,1.7,3.0,.565,.486,1.0,1.4,.724,1.1,3.0,4.1,1.9,0.5,0.3,0.6,1.7,6.4,Brooklyn Nets,44,38,.537,7.0,112.9,112.1,0.82,2077.0,4.033566,2.872423,2.886426,2.391358,3.515848,2.984951,0.335464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14777,NaN,Marcus Williams,NaN,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112.0,-2.292752,-6.779533,-0.111628,-0.111628,-3.505993,-8.006671,0.510876
14778,NaN,Marcus Williams,NaN,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,-2.557043,11.732674,0.022832,0.022832,-0.725563,11.260487,2.782292
14779,NaN,Dorell Wright,NaN,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,-4.860210,8.689018,0.114297,0.000000,-9.277933,-0.079499,-0.056953
14780,NaN,Trevon Scott,NaN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,8.111026,2.149058,0.026743,0.026743,9.268916,3.487452,-2.734333


In [125]:
#Let's see how many null values we have in our completed_df
completed_df.isnull().sum()


Rank                83
Player               0
Share               83
Year                 0
Pos                 83
Age                 83
Tm                  83
G                   83
GS                  83
MP                  83
FG                  83
FGA                 83
FG%                 83
3P                  83
3PA                 83
3P%                 83
2P                  83
2PA                 83
2P%                 83
eFG%                83
FT                  83
FTA                 83
FT%                 83
ORB                 83
DRB                 83
TRB                 83
AST                 83
STL                 83
BLK                 83
TOV                 83
PF                  83
PTS                 83
Team                83
W                   83
L                   83
W/L%                83
GB                  83
PS/G                83
PA/G                83
SRS                 83
poss                64
raptor_defense      64
raptor_total        64
war_total  

In [126]:
#Looking through the completed_df, we can see that there are a 102 columns with some null values. Looking through them its obvious that the null values are from players that barely played in the NBA.
#We will drop their data because it have no insight for our analysis as no player that barely played in the NBA will win DPOY
#We will drop any column that has null in the "Rank" column
completed_df.dropna(subset=["Rank"], inplace=True)
completed_df.tail(500)

,Rank,Player,Share,Year,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,poss,raptor_defense,raptor_total,war_total,war_reg_season,predator_defense,predator_total,pace_impact
14199,0,Jaylen Nowell,0.0,2021,SG,21,MIN,42,0,18.1,3.4,8.0,.424,1.2,3.6,.333,2.2,4.4,.497,.499,1.1,1.3,.818,0.5,1.8,2.3,1.5,0.5,0.3,0.7,1.7,9.0,Minnesota Timberwolves,23,49,.319,29.0,112.1,117.7,-5.25,1640.0,-2.238501,-3.125652,-0.145392,-0.145392,-1.698329,-3.062285,-0.052725
14200,0,Josh Okogie,0.0,2021,SG,22,MIN,59,37,20.3,1.7,4.3,.402,0.5,1.8,.269,1.2,2.5,.500,.459,1.4,1.8,.769,1.1,1.6,2.6,1.1,0.9,0.5,0.7,2.1,5.4,Minnesota Timberwolves,23,49,.319,29.0,112.1,117.7,-5.25,2567.0,1.748326,-0.573121,1.333432,1.333432,2.505827,-0.138287,0.303159
14201,0,Naz Reid,0.0,2021,C,21,MIN,70,15,19.2,4.3,8.3,.523,0.9,2.5,.351,3.5,5.8,.597,.576,1.6,2.4,.693,1.2,3.4,4.6,1.0,0.5,1.1,1.0,2.6,11.2,Minnesota Timberwolves,23,49,.319,29.0,112.1,117.7,-5.25,2920.0,0.883143,-0.545205,1.522113,1.522113,1.478141,0.002495,0.459239
14202,0,Ricky Rubio,0.0,2021,PG,30,MIN,68,51,26.1,2.8,7.3,.388,1.0,3.1,.308,1.9,4.2,.447,.454,1.9,2.2,.867,0.4,2.9,3.3,6.4,1.4,0.1,1.6,2.0,8.6,Minnesota Timberwolves,23,49,.319,29.0,112.1,117.7,-5.25,3815.0,-0.420522,-1.424909,1.202132,1.202132,0.663467,-0.411016,0.349873
14203,0,D'Angelo Russell,0.0,2021,PG,24,MIN,42,26,28.5,6.7,15.5,.431,2.9,7.4,.387,3.8,8.1,.471,.523,2.7,3.5,.765,0.4,2.3,2.6,5.8,1.1,0.4,2.7,1.6,19.0,Minnesota Timberwolves,23,49,.319,29.0,112.1,117.7,-5.25,2596.0,-3.179593,-1.239691,0.933453,0.933453,-2.399729,0.120189,1.299887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14694,0,Trevelin Queen,0.0,2022,SG,24,HOU,10,0,7.4,1.5,3.3,.455,0.9,2.4,.375,0.6,0.9,.667,.591,0.4,0.4,1.000,0.7,0.9,1.6,0.4,0.5,0.1,0.7,0.6,4.3,Houston Rockets,20,62,.244,36.0,109.7,118.2,-8.26,163.0,-1.435701,1.223831,0.152545,0.152545,-2.740698,-1.311932,1.675328
14695,0,Dennis Schroder,0.0,2022,SG-PG,28,HOU,64,29,28.7,4.9,11.3,.431,1.3,3.9,.344,3.5,7.4,.477,.490,2.5,2.9,.853,0.5,2.8,3.3,4.6,0.8,0.1,2.2,2.4,13.5,Houston Rockets,20,62,.244,36.0,109.7,118.2,-8.26,3755.0,-0.870595,-2.877334,-0.117276,-0.117276,-0.305205,-1.947502,-0.366924
14696,0,Alperen Sengun,0.0,2022,C,19,HOU,72,13,20.7,3.5,7.3,.474,0.4,1.6,.248,3.1,5.7,.539,.502,2.3,3.2,.711,1.9,3.5,5.5,2.6,0.8,0.9,2.0,3.0,9.6,Houston Rockets,20,62,.244,36.0,109.7,118.2,-8.26,3155.0,-1.267561,-2.308609,0.338049,0.338049,-1.325915,-2.984400,0.814098
14697,0,Jae'Sean Tate,0.0,2022,PF,26,HOU,78,77,26.4,4.7,9.3,.498,0.8,2.6,.312,3.8,6.8,.569,.541,1.7,2.4,.707,1.7,3.7,5.4,2.8,0.9,0.5,1.8,3.7,11.8,Houston Rockets,20,62,.244,36.0,109.7,118.2,-8.26,4367.0,0.775755,-0.529578,2.337336,2.337336,0.612565,-1.138323,0.351132


In [127]:
completed_df.isnull().sum()

Rank                 0
Player               0
Share                0
Year                 0
Pos                  0
Age                  0
Tm                   0
G                    0
GS                   0
MP                   0
FG                   0
FGA                  0
FG%                  0
3P                   0
3PA                  0
3P%                  0
2P                   0
2PA                  0
2P%                  0
eFG%                 0
FT                   0
FTA                  0
FT%                  0
ORB                  0
DRB                  0
TRB                  0
AST                  0
STL                  0
BLK                  0
TOV                  0
PF                   0
PTS                  0
Team                 0
W                    0
L                    0
W/L%                 0
GB                   0
PS/G                 0
PA/G                 0
SRS                  0
poss                64
raptor_defense      64
raptor_total        64
war_total  

In [128]:
#Find "Nikola Vucevic" in the df
completed_df[completed_df["Player"]=="Muggsy Bogues"]

,Rank,Player,Share,Year,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team,W,L,W/L%,GB,PS/G,PA/G,SRS,poss,raptor_defense,raptor_total,war_total,war_reg_season,predator_defense,predator_total,pace_impact
485,0,Muggsy Bogues,0.0,1995,PG,30,CHH,78,78,33.7,4.5,9.4,.477,0.1,0.4,.200,4.4,9.0,.489,.481,2.1,2.3,.889,0.7,2.6,3.3,8.7,1.3,0.0,1.7,1.9,11.1,Charlotte Hornets,50,32,.610,2.0,100.6,97.3,2.87,5311.0,0.094620,2.728908,7.643739,7.734863,0.420015,2.530030,-0.884164
5629,0,Muggsy Bogues,0.0,1991,PG,26,CHH,81,46,28.4,3.0,6.5,.460,0.0,0.1,.000,3.0,6.3,.471,.460,1.1,1.3,.796,0.7,2.0,2.7,8.3,1.7,0.0,1.5,2.0,7.0,Charlotte Hornets,26,56,.317,35.0,102.8,108.0,-4.95,4714.0,0.470436,3.287739,7.047789,7.047789,0.781983,3.088243,-0.478330
5806,0,Muggsy Bogues,0.0,1992,PG,27,CHH,82,69,34.0,3.9,8.2,.472,0.0,0.3,.074,3.8,7.9,.489,.474,1.1,1.5,.783,0.7,2.2,2.9,9.1,2.1,0.1,1.9,1.9,8.9,Charlotte Hornets,31,51,.378,36.0,109.5,113.4,-3.57,5979.0,0.198609,1.957192,6.667173,6.667173,0.576071,2.041537,-0.486478
6260,0,Muggsy Bogues,0.0,1993,PG,28,CHH,81,80,35.0,4.1,9.0,.453,0.1,0.3,.231,4.0,8.7,.462,.458,1.7,2.1,.833,0.6,3.0,3.7,8.8,2.0,0.1,1.9,2.2,10.0,Charlotte Hornets,44,38,.537,13.0,110.1,110.4,-0.02,6652.0,0.957490,2.910422,9.206217,7.830303,1.485710,3.328787,-0.482620
6522,0,Muggsy Bogues,0.0,1994,PG,29,CHH,77,77,35.7,4.6,9.8,.471,0.0,0.2,.167,4.6,9.6,.476,.473,1.6,2.0,.806,1.0,3.1,4.1,10.1,1.7,0.0,2.2,1.9,10.8,Charlotte Hornets,41,41,.500,16.0,106.5,106.7,-0.23,5642.0,-0.221327,2.797961,7.708204,7.708204,0.324642,3.054810,-0.799123
6930,0,Muggsy Bogues,0.0,1996,PG,31,CHH,6,0,12.8,1.0,2.7,.375,0.0,0.2,.000,1.0,2.5,.400,.375,0.3,0.3,1.000,1.0,0.2,1.2,3.2,0.3,0.0,1.0,0.7,2.3,Charlotte Hornets,41,41,.500,31.0,102.8,103.4,-0.48,149.0,-3.222421,-4.062928,-0.050917,-0.050917,-4.294241,-5.776385,-1.193289
7267,0,Muggsy Bogues,0.0,1997,PG,32,CHH,65,65,28.9,3.1,6.8,.460,0.9,2.2,.417,2.2,4.6,.482,.528,0.8,1.0,.844,0.4,1.8,2.2,7.2,1.3,0.0,1.7,1.8,8.0,Charlotte Hornets,54,28,.659,15.0,98.9,97.0,2.13,3616.0,-0.562055,2.516038,5.170190,5.123815,-0.498679,2.275118,-0.645871
7794,0,Muggsy Bogues,0.0,1998,PG,33,GSW,61,31,25.7,2.3,5.3,.437,0.1,0.3,.250,2.2,5.0,.446,.443,1.0,1.1,.897,0.5,1.7,2.2,5.4,1.1,0.0,1.7,1.0,5.7,Golden State Warriors,19,63,.232,42.0,88.3,97.4,-9.20,3023.0,-0.678948,-0.724558,1.609669,1.609669,-1.049165,-1.939956,-0.876763
8049,0,Muggsy Bogues,0.0,1999,PG,34,GSW,36,5,19.8,2.1,4.3,.494,0.0,0.2,.000,2.1,4.1,.514,.494,0.9,1.0,.861,0.4,1.6,2.0,3.7,1.2,0.0,1.3,1.2,5.1,Golden State Warriors,21,29,.420,14.0,88.3,90.8,-2.63,1338.0,0.995084,1.774099,1.640869,1.640869,0.538405,0.672908,-0.392222
8341,0,Muggsy Bogues,0.0,2000,PG,35,TOR,80,5,21.6,2.0,4.5,.439,0.2,0.6,.333,1.8,3.8,.456,.462,1.0,1.1,.908,0.3,1.4,1.7,3.7,0.8,0.1,0.7,1.5,5.1,Toronto Raptors,45,37,.549,11.0,97.2,97.3,-0.46,3532.0,-1.068611,-0.719024,1.862030,1.913199,-1.126649,-1.399587,-0.507711


In [129]:
#Let's save completed_df as completed.csv in the "Cleaned" folder
completed_df.to_csv('/Users/david/Documents/Python Projects/DPOY/Cleaned/completed.csv')
